# NL2SQL Pipeline Demo - Azure AI Agent Service

## Overview
This notebook demonstrates the **Azure AI Agent Service** implementation of our Natural Language to SQL pipeline.

### Key Features
- 🤖 **Persistent AI Agents** - Agents remain in Azure AI Foundry (not deleted after use)
- 🔐 **Enterprise Authentication** - Uses DefaultAzureCredential (Azure CLI, Managed Identity)
- 📊 **Built-in Observability** - Native Azure AI Foundry tracing
- ⚡ **Performance** - Agent reuse eliminates creation overhead
- 🎯 **2-Agent Architecture** - Intent extraction + SQL generation

### Pipeline Flow
```
Natural Language Query
         ↓
    [Intent Agent] ← Persistent agent in Azure AI Foundry
         ↓
    Intent JSON (entities, filters, metrics)
         ↓
    [SQL Agent] ← Persistent agent with schema context
         ↓
    Generated T-SQL
         ↓
    SQL Execution (Azure SQL)
         ↓
    Formatted Results + Token Usage + Cost
```

### Comparison with LangChain
| Feature | LangChain | Azure AI Agent Service |
|---------|-----------|------------------------|
| Orchestration | Prompt chains | Agents + Threads |
| Authentication | API Key | DefaultAzureCredential |
| State | Stateless | Stateful threads |
| Persistence | No | Yes (Azure AI Foundry) |
| Dependencies | 3 packages | 2 packages |

---

**Let's explore the pipeline step by step!**

## Step 1: Import Required Libraries

We'll import the Azure AI Agent Service SDK and other necessary modules.

In [ ]:
import os
import sys
import json
import time
from datetime import datetime
from dotenv import load_dotenv

# Azure AI Agent Service
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Add current directory to path for local imports
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

print("✓ All libraries imported successfully!")
print(f"  - azure-ai-projects: Available")
print(f"  - azure-identity: Available")
print(f"  - Python version: {sys.version.split()[0]}")

## Step 2: Load Environment Configuration

Load Azure AI Foundry project configuration and verify settings.

In [ ]:
# Load environment variables
load_dotenv()

# Azure AI Foundry configuration
PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME")

# Azure SQL configuration
AZURE_SQL_SERVER = os.getenv("AZURE_SQL_SERVER")
AZURE_SQL_DB = os.getenv("AZURE_SQL_DB")

print("Environment Configuration:")
print("=" * 60)
print(f"✓ Azure AI Project: {PROJECT_ENDPOINT}")
print(f"✓ Model Deployment: {MODEL_DEPLOYMENT_NAME}")
print(f"✓ Azure SQL Server: {AZURE_SQL_SERVER}")
print(f"✓ Database: {AZURE_SQL_DB}")
print("=" * 60)

# Verify required variables
if not PROJECT_ENDPOINT or not MODEL_DEPLOYMENT_NAME:
    raise ValueError("Missing required environment variables. Check .env file!")

## Step 3: Initialize Azure AI Project Client

Connect to Azure AI Foundry using DefaultAzureCredential (Azure CLI authentication).

In [ ]:
# Initialize Azure AI Project Client
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

print("✓ Azure AI Project Client initialized successfully!")
print(f"  Endpoint: {PROJECT_ENDPOINT}")
print(f"  Authentication: DefaultAzureCredential (Azure CLI)")
print("\n📌 Note: This uses your Azure CLI login (az login)")
print("   No API keys stored in code!")

## Step 4: Load Database Schema

Load the CONTOSO-FI database schema that will be provided to the SQL generation agent.